'''
<br><br>
@Author: Shivraj Yelave<br>
@Date: 13-09-24<br>
@Last modified by: Shivraj Yelave<br>
@Last modified time: 13-09-24<br>
@Title: CURD Operation in Dynamodb Using boto3<br><br>
'''

In [2]:
import boto3
from dotenv import load_dotenv
import os 

load_dotenv()



True

In [6]:

def initialize_dynamodb(region, access_key_id, secret_access_key):
    """
    Initializes a session with Amazon DynamoDB.

    Parameters:
    region (str): The AWS region where the DynamoDB instance is hosted (e.g., 'ap-south-1').
    access_key_id (str): AWS access key ID for authentication.
    secret_access_key (str): AWS secret access key for authentication.

    Returns:
    dynamodb (DynamoDB.ServiceResource): A DynamoDB resource object.
    """
    dynamodb = boto3.resource(
        'dynamodb',
        region_name=region,
        aws_access_key_id=access_key_id,
        aws_secret_access_key=secret_access_key
    )
    return dynamodb

def get_table_status(dynamodb, table_name):
    """
    Accesses the specified DynamoDB table and retrieves its status.

    Parameters:
    dynamodb (DynamoDB.ServiceResource): A DynamoDB resource object.
    table_name (str): The name of the DynamoDB table.

    Returns:
    str: The status of the DynamoDB table (e.g., 'ACTIVE').
    """
    table = dynamodb.Table(table_name)
    return table.table_status

def main():
    """
    The main function that initializes DynamoDB, accesses a table, and prints its status.
    """
    region = 'ap-south-1'
    access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    
    # Initialize DynamoDB session
    dynamodb = initialize_dynamodb(region, access_key_id, secret_access_key)
    
    # Specify the table name
    table_name = 'Shiv'
    
    # Get table status
    table_status = get_table_status(dynamodb, table_name)
    
    # Print the table status
    print(f"Table status: {table_status}")

if __name__ == '__main__':
    main()


Table status: ACTIVE


In [7]:
import boto3
import os

def create_dynamodb_table(dynamodb, table_name):
    """
    Creates a DynamoDB table with specified key schema, attribute definitions, and provisioned throughput.

    Parameters:
    dynamodb (DynamoDB.ServiceResource): A DynamoDB resource object.
    table_name (str): The name of the table to be created.

    Returns:
    table (DynamoDB.Table): The created DynamoDB table object.
    """
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': 'username',
                'KeyType': 'HASH'  # Partition key
            },
            {
                'AttributeName': 'email',
                'KeyType': 'RANGE'  # Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'username',
                'AttributeType': 'S'  # String type
            },
            {
                'AttributeName': 'email',
                'AttributeType': 'S'  # String type
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
    # Wait for the table to be created before proceeding
    table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
    return table

def main():
    """
    The main function to initialize DynamoDB and create a 'users' table.
    """
    region = 'ap-south-1'
    access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

    # Initialize DynamoDB session
    dynamodb = boto3.resource(
        'dynamodb',
        region_name=region,
        aws_access_key_id=access_key_id,
        aws_secret_access_key=secret_access_key
    )

    # Specify the table name
    table_name = 'users'

    # Create the DynamoDB table
    table = create_dynamodb_table(dynamodb, table_name)

    # Print the table status
    print(f"Table '{table_name}' created with status: {table.table_status}")

if __name__ == '__main__':
    main()


In [15]:
def add_item_to_table(dynamodb, table_name, item):
    """
    Adds a new item to the specified DynamoDB table.

    Parameters:
    dynamodb (DynamoDB.ServiceResource): A DynamoDB resource object.
    table_name (str): The name of the DynamoDB table.
    item (dict): The item to be added to the table.

    Returns:
    None
    """
    table = dynamodb.Table(table_name)
    table.put_item(Item=item)

def main():
    """
    The main function that initializes DynamoDB, accesses a table, adds an item, and prints the table status.
    """
    region = 'ap-south-1'
    access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    
    # Initialize DynamoDB session
    dynamodb = initialize_dynamodb(region, access_key_id, secret_access_key)
    
    # Specify the table name
    table_name = 'Shiv'
    
    # Get table status
    table_status = get_table_status(dynamodb, table_name)
    print(f"Table status: {table_status}")

    # Add a new item to the table
    item = {
        'username': 'johndoe',
        'email': 'johndoe@example.com',
        'age': 30
    }
    add_item_to_table(dynamodb, table_name, item)

if __name__ == '__main__':
    main()

{'ResponseMetadata': {'RequestId': 'FD6NJLM60HF5IFME0H9UDEBGBJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 13 Sep 2024 10:00:42 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'FD6NJLM60HF5IFME0H9UDEBGBJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [9]:
response = table.get_item(
    Key={
        'username': 'johndoe',
        'email': 'johndoe@example.com'
    }
)
item = response.get('Item')
print(item)


{'email': 'johndoe@example.com', 'username': 'johndoe', 'age': Decimal('30')}


In [10]:
def update_item(dynamodb, table_name, key, update_expression, expression_attribute_values):
    """
    Updates an item in a DynamoDB table.

    Parameters:
    dynamodb (DynamoDB.ServiceResource): A DynamoDB resource object.
    table_name (str): The name of the DynamoDB table.
    key (dict): The primary key of the item to update.
    update_expression (str): An expression that defines the update.
    expression_attribute_values (dict): Values for the update expression.

    Returns:
    dict: The updated attributes of the item.
    """
    table = dynamodb.Table(table_name)
    response = table.update_item(
        Key=key,
        UpdateExpression=update_expression,
        ExpressionAttributeValues=expression_attribute_values,
        ReturnValues="UPDATED_NEW"
    )
    return response.get('Attributes')

def main():
    """
    The main function that initializes DynamoDB, updates an item, and prints the updated attributes.
    """
    region = 'ap-south-1'
    access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    
    # Initialize DynamoDB session
    dynamodb = initialize_dynamodb(region, access_key_id, secret_access_key)
    
    # Specify the table name and update details
    table_name = 'Shiv'
    key = {
        'username': 'johndoe',
        'email': 'johndoe@example.com'
    }
    update_expression = "set age = :a"
    expression_attribute_values = {
        ':a': 31
    }
    
    # Update the item
    updated_attributes = update_item(dynamodb, table_name, key, update_expression, expression_attribute_values)
    
    # Print the updated attributes
    print(f"Updated attributes: {updated_attributes}")

if __name__ == '__main__':
    main()

{'Attributes': {'age': Decimal('31')},
 'ResponseMetadata': {'RequestId': '64A74SAOKIJDNRJ41B64T87HQNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 13 Sep 2024 09:55:00 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '33',
   'connection': 'keep-alive',
   'x-amzn-requestid': '64A74SAOKIJDNRJ41B64T87HQNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '377911233'},
  'RetryAttempts': 0}}

In [11]:
def delete_item(dynamodb, table_name, key):
    """
    Deletes an item from the specified DynamoDB table.

    Parameters:
    dynamodb (DynamoDB.ServiceResource): A DynamoDB resource object.
    table_name (str): The name of the DynamoDB table.
    key (dict): A dictionary specifying the primary key of the item to delete.

    Returns:
    dict: The response from the delete_item operation.
    """
    table = dynamodb.Table(table_name)
    response = table.delete_item(Key=key)
    return response

def main():
    """
    The main function that initializes DynamoDB, accesses a table, prints its status, and deletes an item.
    """
    region = 'ap-south-1'
    access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    
    # Initialize DynamoDB session
    dynamodb = initialize_dynamodb(region, access_key_id, secret_access_key)
    
    # Specify the table name
    table_name = 'Shiv'
    
    # Get table status
    table_status = get_table_status(dynamodb, table_name)
    print(f"Table status: {table_status}")
    
    # Specify the key of the item to delete
    key = {
        'username': 'johndoe',
        'email': 'johndoe@example.com'
    }
    
    # Delete the item
    response = delete_item(dynamodb, table_name, key)
    print(f"Delete item response: {response}")

if __name__ == '__main__':
    main()

{'ResponseMetadata': {'RequestId': 'SVSANP4J9QHKIBQOVR0BL7JM7RVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 13 Sep 2024 09:56:34 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'SVSANP4J9QHKIBQOVR0BL7JM7RVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [12]:
def batch_write_items(table):
    """
    Batch writes items to the specified DynamoDB table.

    Parameters:
    table (DynamoDB.Table): A DynamoDB Table resource object.
    """
    with table.batch_writer() as batch:
        batch.put_item(Item={'username': 'user1', 'email': 'user1@example.com', 'age': 25})
        batch.put_item(Item={'username': 'user2', 'email': 'user2@example.com', 'age': 28})

def main():
    """
    The main function that initializes DynamoDB, accesses a table, performs batch writes, and prints the table status.
    """
    region = 'ap-south-1'
    access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    
    # Initialize DynamoDB session
    dynamodb = initialize_dynamodb(region, access_key_id, secret_access_key)
    
    # Specify the table name
    table_name = 'Shiv'
    
    # Access the table
    table = dynamodb.Table(table_name)
    
    # Perform batch write operations
    batch_write_items(table)
    
    # Get table status
    table_status = get_table_status(dynamodb, table_name)
    
    # Print the table status
    print(f"Table status: {table_status}")

if __name__ == '__main__':
    main()


In [16]:
import csv

def export_table_to_csv(dynamodb, table_name, csv_filename):
    """
    Exports data from a DynamoDB table to a CSV file.

    Parameters:
    dynamodb (DynamoDB.ServiceResource): A DynamoDB resource object.
    table_name (str): The name of the DynamoDB table.
    csv_filename (str): The name of the CSV file to write the data to.
    """
    # Access the DynamoDB table
    table = dynamodb.Table(table_name)
    
    # Scan the table to get all items
    response = table.scan()
    items = response['Items']
    
    # Open the CSV file for writing
    with open(csv_filename, 'w', newline='') as csvfile:
        # Define fieldnames based on the expected columns in the DynamoDB items
        fieldnames = ['username', 'email', 'age']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write the header row
        writer.writeheader()
        
        # Write each item to the CSV file
        for item in items:
            writer.writerow(item)
            print(item)

def main():
    """
    The main function that initializes DynamoDB, exports data to CSV, and prints the items.
    """
    region = 'ap-south-1'
    access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    table_name = 'Shiv'
    csv_filename = 'export.csv'
    
    # Initialize DynamoDB session
    dynamodb = initialize_dynamodb(region, access_key_id, secret_access_key)
    
    # Export table data to CSV
    export_table_to_csv(dynamodb, table_name, csv_filename)

if __name__ == '__main__':
    main()


{'email': 'user1@example.com', 'username': 'user1', 'age': Decimal('25')}
{'email': 'johndoe@example.com', 'username': 'johndoe', 'age': Decimal('30')}
{'email': 'user2@example.com', 'username': 'user2', 'age': Decimal('28')}
